<a href="https://colab.research.google.com/github/alex283h/YOLOv3_test_task/blob/master/task_1_230622.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Устанавливаем пакет для загрузки рыночных данных:

In [ ]:
!pip3 install yfinance 

Также установим пакет для расчета различных индикаторов рынка:

In [ ]:
!pip3 install ta

Все индикаторы, которые могут быть рассчитаны с помощью данного пакета можно посмотреть по ссылке:

https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#volume-indicators

Например, в статье на которую мы ориентируемся индикатор ATR был реализован собственноручно. Насколько правильно неизвестно, кода-то нет. Здесь же все выверено и есть много дополнительных полезных "штук" приработе с которовками.
Поэтому в качестве первого нашего улучшения будем пользоватсья данным пакетом.
Удобьно и быстро.

Загрузим все необходимые библиотеки для последующей работы:

In [ ]:
import pandas as pd # для работы с дата-фреймами
import numpy as np # для работы с массивами и матрицами
from matplotlib import pyplot as plt # для работы с графиками
import datetime # для работы с временем и датами
import yfinance as yf # для загрузки котировок
import seaborn as sns # для работы с графиками
from sklearn.cluster import KMeans # для кластеризации
import ta # индикаторы технического анализа
from sklearn.preprocessing import StandardScaler,MinMaxScaler # масштабирование признаков
from yellowbrick.cluster import KElbowVisualizer # удобная визуализация кластерной модели с различным числом кластеров
import plotly.graph_objects as go # визуализация рыночных показателей и индикаторов
from sklearn.decomposition import PCA # наиболее популярный метод сокращения размерности для визуализации
from sklearn.manifold import TSNE # более продвинутый метод сокращения размерности для визуализации

Обределим даты начала и окончания загрузки показателя SPX (еще S&P 500 или GSPC; именно он выбран в статье за основу).
Загрузим днные за примерно 32 года по дням:

In [ ]:
end_time = datetime.date.today()
start_time = end_time - datetime.timedelta(11850)

In [ ]:
SPX = yf.download('^GSPC',start = start_time, end = end_time)
SPX

Данные загружены, все хорошо. Теперь дополнительно, как и авторы статьи загрузим вспомогательный индикатор-показатель рынка VIX (за тоже время):

In [ ]:
VIX = yf.download('^VIX',start = start_time, end = end_time).Close
VIX

Визуализируем этот показатель:

In [ ]:
fig = go.Figure(data=go.Scatter(x=VIX.index,y=VIX, mode='lines'))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

VIX — это индикатор волатильности американской экономики. Он показывает, будет ли индекс S&P 500 волатилен в будущем. Когда VIX растет, это значит, что инвесторы ожидают сильных колебаний фондового рынка и даже наступления кризиса: именно к таким последствиям может привести рост волатильности S&P 500. Хорошо то, что данный показатель не имеет явно выраженного тренда и является стационарным относительно некоего среднего значения. То есть его диапазон возможных значений конечен и псевдо-фиксирован.

Далее вычислим показатель ATR за период равный 14 дням. Как и у авторов статьи:

In [ ]:
ATR = ta.volatility.AverageTrueRange(SPX.High,SPX.Low,SPX.Close,window = 14, fillna = True).average_true_range()/SPX.Close
ATR

Визуализируем показатель:

In [ ]:
fig = go.Figure(data=go.Scatter(x=ATR.index,y=ATR, mode='lines'))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

В целом визуально (возможно и по статистикам) данный показатель похож на предыдущий.

Вычислим (как и у авторов статьи) следующий показатель - Daily retutns:

In [ ]:
DR = ta.others.DailyReturnIndicator(SPX.Close, fillna = True).daily_return()
DR[0] = DR.mean() # здесь важный момент! чтобы самое первое значение не было слишком большим и отличающимся от остальных, заменим его на среднее (специфика вычислений показателя)
DR

Визуализируем показатель:

In [ ]:
fig = go.Figure(data=go.Scatter(x=DR.index,y=DR, mode='lines'))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Выглядит отлично. Как минимум стационарно по среднему (среднее около 0). И нестационарно по дисперсии. Для кластеризации что надо!

Далее авторы также применяют показатель Return Volume. Его нет в пакетной врсии, поэтому просто воспользуемся готовым авторским кодом:

In [ ]:
RV = SPX.Volume/SPX.Volume.rolling(40).mean()
RV = RV.fillna(0)
RV

И визуализация:

In [ ]:
fig = go.Figure(data=go.Scatter(x=RV.index,y=RV, mode='lines'))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Вполне годится. Стационарность относительно среднего есть.

На этих 4 показателях авторы статьи останавливаются и проделывают кластеризацию. Мы же в качестве второго улучшения добавим на 1 показатель больше. Метод кластеризации KMeans может неплохо справляться где-то с 8-10 показателями (индикаторами). Далее начинается проблема "прколятия размерности".

Выберем в качестве такого показателя Stochastic RSI. Важен не столько смысл данного показателя, как тот факт, что показатель стауионарен относительно среднего и ограничен в диапазоне значений от 0 до 1.

Если Вы работаете с рынками и знаете какие показатели лучше применить для данной задачи по смыслу (кластеризация рыночных моментов) и эти показатели стационарны относительно среднего - можно смело эксперементировать!

В рамках кластеризации плохи те показатели, что имеют тренды как и сам рынок. Такие данные будут очень плохо кластеризуемы.


In [ ]:
RSI = ta.momentum.StochRSIIndicator(SPX.Close, fillna = True).stochrsi()
RSI

Визуализация:

In [ ]:
fig = go.Figure(data=go.Scatter(x=RSI.index,y=RSI, mode='lines'))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Отлично! Теперь соберем все вычисленные показатели в единый дата-фрейм X для последующей кластеризации:

In [ ]:
X = pd.concat([DR,ATR,VIX,RV,RSI], axis=1)
X.columns = ['DR','ATR','VIX','RV','RSI']
X

Все хорошо. Но бывают ситуации, когда данные основного показателя (в нашем случае SPX) приходят раньше, чем VIX и мы можем получить пропущенные значения в последней строке. Сейчас все хорошо. Но для корректировки таких случаев добавим следующий код:

In [ ]:
X = X.dropna()
X

Следующим важным улучшением (чего нет у авторов!) является масштабирование всех признаков. Метод K-Means чувствителен к диапазону индикаторов, так как основывается на вычислениях расстояний между векторами. Все индикаторы должны быть приведены к единому диапазону, иначе какой-то один будет "доминантным", а остальные могут не оказывать почти никакого влияния. Работа с ненормированными (немасштабированными) данными частая ошибка при кластеризации! Вот подробные объяснения почему: https://medium.com/analytics-vidhya/why-is-scaling-required-in-knn-and-k-means-8129e4d88ed7

Произведем масштабирование всех прихнаков в интервале 0-1:

In [ ]:
t_scaler = MinMaxScaler().fit(X)
t_X = pd.DataFrame(t_scaler.transform(X))
t_X.columns = X.columns
t_X

Теперь все хорошо. Но перед тем, как начинать какую-либо кластеризацию, всегда стоит посмотреть на распределение показателей в 2D проекции. Для этого существует множество методов: PCA, tSNE, UMAP... Мы воспользуемся наиболее простым и популярным PCA и визуализируем первые 2 компоненты в 2D плоскости. Если мы увидем какие-то отдельные группы точек - то хорошо. Если нет - то вообще кластеризация, как таковая, и её смысл может быть весьма спорным... Сделаем это:

In [ ]:
pca = PCA(n_components = 2)
pca_data = pd.DataFrame(pca.fit_transform(t_X),columns=['PC1','PC2']) 
sns.scatterplot(x="PC1",y="PC2",data=pca_data)
plt.show()

Честно говоря, мало обещающий график. Каких-либо групп точек мы не видем. Как разделить или кластеризовать такие данные? Однозначный способ не прослеживается.
..
Бывает так, что PCA в двухмерном пространстве действительно не отображет сложные многомерные кластеры. Попробуем применить более продвинутый метод - tSNE. Он также основан на вычислении расстояний, как и метод кластеризации K-Means. Но при этом учитывает многомерные статистики. Метод требует куда как больше времени на вычисления (чем больше значение n_iter - тем дольше и точнее будут кластеры; можно экспериментировать!).

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=0, perplexity = 15, n_iter = 1500)
tsne_data = pd.DataFrame(tsne.fit_transform(t_X),columns=['PC1','PC2']) 
sns.scatterplot(x="PC1",y="PC2",data=tsne_data)
plt.show()

Какие-то группы точек уже можно различать. Но и тут нет красивой однозначности. Поэтому, попробуем сделать кластеризацию не только исходных данных с 5-ю показателями, но и кластеризацию данных, полученных после метода tSNE. ВОзможно будет лучше.

В следующих шагах мы выполним K-Means кластеризацию исходных данных и найдем "оптимальное" число кластеров. Авторы применяют в статье лишь один метод - метод "локтя" или Elbow метод. Но есть и другие метрики для првоерки оптимального числа кластеров. Наиболее популярные три:

- Elbow;
- Calinski Harabasz;
- Silhouette.

Вот и посмотрим их все. Сколько кластеров оптимально получится используя ту или иную метрику. Это будет нашим еще одним улучшением.

**1. Кластеризация по исходным данным.**

In [ ]:
# Elbow метод для K-Means
model = KMeans()
# k - диапазон возможных кластеров
visualizer = KElbowVisualizer(model, k=(2,16), timings= True)
visualizer.fit(t_X)
# визуализация
plt.show()
visualizer.show()

Вот с помощью данного метода получилось 5 кластеров - лучший выбор.
Визуализируем эти кластеры с помощью 2D проекции на базе tSNE (он получше все сделал, как мы видели ранее):

In [ ]:
m1 = KMeans(n_clusters=5).fit(t_X)
tsne_data['m1'] = pd.Categorical(m1.labels_) # добавим метки кластеров в данные tsne
sns.scatterplot(x="PC1",y="PC2",hue="m1",data=tsne_data)
plt.show()

Аак-то вот так распределились облака точек. Проверим другие два метода:

In [ ]:
# Calinski Harabasz метод
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16),metric='calinski_harabasz', timings= True)
visualizer.fit(t_X)
visualizer.show()

Здесь метод "посчитал", что 2 кластера - лучший выбор.

In [ ]:
m2 = KMeans(n_clusters=2).fit(t_X)
tsne_data['m2'] = pd.Categorical(m2.labels_) # добавим метки кластеров в данные tsne
sns.scatterplot(x="PC1",y="PC2",hue="m2",data=tsne_data)
plt.show()

Тут разделение пошло по границе наименьшего "соприкосновения" 2 облаков точек. Хорошо это или нет, сказать пока сложно. Взглянем на последний метод:

In [ ]:
# Silhouette
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16),metric='silhouette', timings= True)
visualizer.fit(t_X)
visualizer.show()

И этот метод "рекомендует" выбрать только 2 кластера.

У авторов статьи число кластеров получилось больше двух. Но они руководствовались лишь одним методом. В нашем же случае из 3-х проверенных методов 2 "говорят" в пользу выбора только 2 кластеров. Нам следует так и сделать... это более надежный подход.

Теперь в заключение следует вывести исходный график SPX (данные по Close) с подсветкой двух кластеров:

In [ ]:
fig = go.Figure(data=go.Scatter(x=SPX.index,y=SPX.Close, 
                                mode="markers",
                                marker_color=m2.labels_))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Вот такой график получился. Да, похоже на то что сильным медвежьим трендам более соответствуют последовательности "синих" кластеров, а "желтые" говорят о равномерном движении.

Если же "подсветить" кластеры, что были найдены первым методом (их 5), то получим вот что:

In [ ]:
fig = go.Figure(data=go.Scatter(x=SPX.index,y=SPX.Close, 
                                mode="markers",
                                marker_color=m1.labels_))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Здесь явно прослеживаются "ораньжевые" кластеры.

Все это можно интерпретировать как нам угодно, но нужно не уйти в сторону и не "выдавать желаемое за действительное".

**2. Кластеризация по данным tSNE.**

In [ ]:
# Elbow метод для K-Means
model = KMeans()
# k - диапазон возможных кластеров
visualizer = KElbowVisualizer(model, k=(2,16), timings= True)
visualizer.fit(tsne_data.iloc[:,:2])
# визуализация
plt.show()
visualizer.show()

Все теже 5 кластеров! Визуализация:

In [ ]:
m3 = KMeans(n_clusters=5).fit(tsne_data.iloc[:,:2])
tsne_data['m3'] = pd.Categorical(m3.labels_) # добавим метки кластеров в данные tsne
sns.scatterplot(x="PC1",y="PC2",hue="m3",data=tsne_data)
plt.show()

Сейчас красивее и четче видны "облака" точек.

Второй метод:

In [ ]:
# Calinski Harabasz метод
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16),metric='calinski_harabasz', timings= True)
visualizer.fit(tsne_data.iloc[:,:2])
visualizer.show()

А тут метод "предпочел" выбрать 6 кластеров!

Визуализируем:

In [ ]:
m3 = KMeans(n_clusters=6).fit(tsne_data.iloc[:,:2])
tsne_data['m4'] = pd.Categorical(m3.labels_) # добавим метки кластеров в данные tsne
sns.scatterplot(x="PC1",y="PC2",hue="m4",data=tsne_data)
plt.show()

Выглядит вполне себе неплохо. И првоерим последний метод:

In [ ]:
# Silhouette
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16),metric='silhouette', timings= True)
visualizer.fit(tsne_data.iloc[:,:2])
visualizer.show()

А этот метод предлагает выбрать только 3 группы! Визуализируем их:

In [ ]:
m5 = KMeans(n_clusters=3).fit(tsne_data.iloc[:,:2])
tsne_data['m5'] = pd.Categorical(m5.labels_) # добавим метки кластеров в данные tsne
sns.scatterplot(x="PC1",y="PC2",hue="m5",data=tsne_data)
plt.show()

Тут выбрать сложно, поэтому посмотрим на реальный график цены SPX со всеми обозначенными кластерами кластерами для каждого из 3-х методов поиска оптимального числа кластеров:

In [ ]:
fig = go.Figure(data=go.Scatter(x=SPX.index,y=SPX.Close, 
                                mode="markers",
                                marker_color=m3.labels_))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(x=SPX.index,y=SPX.Close, 
                                mode="markers",
                                marker_color=m4.labels_))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(x=SPX.index,y=SPX.Close, 
                                mode="markers",
                                marker_color=m5.labels_))
fig.update_layout(autosize=False,width=800,height=500)
fig.show()

Самый первый график интереснее. Нисходящие тенденции более оранжево-красные, а восходящие розово-синии.

**Рекомендации**

Не смотря на то, что было проверено несколько методов поиска оптимального числа кластеров по 5 показателям и по их проекции в 2D плоскость методом tSNE существенных результатов, которые бы были видны визуально не видно. Не вижно это явно и в статье авторов (у них признаки не нормируются). Скорее можно говорить о некоторых незначительных закономерностях которые "подсвечиваются" метками кластеров.

Как альтернативу работе, можно попробовать проводить кластеризацию рыночных паттернов, то есть последовательностей рыночных цен, например, взяв их понедельно. Сюда также можно добавить технические индикаторы для "большей" информативности.

Но и в том и в другом можно построить модель классификации будущего значения рынка (то есть модель, которая бы прогнозировала кластер к которому будет принадлежать будущее значение или паттерн).

